In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io

from keras.models import Sequential
from keras.models import model_from_json
from keras.layers import Dense, Conv2D, Flatten
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.


# Deep Learning study on the results of the 1D Pseudo-Wigner Distribution using Neural Networks

**Why?**

Check if the wigner distribution of an hologram is capable to give us enough information to be able to predict how many point sources generated the hologram (1 to 5 sources).

**How?**

Using a Convolutional Neural Networks (CNN) to solve this classification problem.

**What?**

Using the keras libray (python).

**Some examples:**
* https://towardsdatascience.com/building-a-convolutional-neural-network-cnn-in-keras-329fbbadc5f5 

### Load dataset

In [2]:
%%time

path = 'output/wigner_distribution/'
file_name = 'wd_results.npy'

dataset = np.load(path + file_name)
print(dataset.shape)

print('Total number of holograms: ' + str(dataset.shape[0]))
print('Number of holograms per class: ' + str(int(dataset.shape[0]/ 5)))

(125, 8, 200, 200)
Total number of holograms: 125
Number of holograms per class: 25
Wall time: 232 ms


## CNN (Convolutional Neural Networks)

### Data pre-processing

In [3]:
def compute_targets_array(nb_class, X_train):
    """
    Compute an array with the targets of the dataset. Note that the number on the array correspond to the number of 
    sources minus one. E.g. Y_array = 1, the number of point sources is 2.
    """
    # Number of the examples
    nb_holograms = X_train.shape[0]
    
    # Number of examples per class
    nb_holograms_class = int(nb_holograms / nb_class)
    
    # Y vector
    Y_array = np.zeros((nb_holograms,))
    counter = 1
    target = 0
    
    for i in range(nb_holograms):
        if counter == (nb_holograms_class + 1):
            target = target + 1
            counter = 1
        Y_array[i,] = target
        counter = counter + 1    
    
    return Y_array

In [4]:
# Select one of the 8 frequencies ! BUG !!!!!!!!!!!!
X_train = dataset[:,0,:,:]

# The 1 signify that the images are greyscale
X_train = X_train.reshape(X_train.shape[0], 200, 200,1)

print(X_train.shape)

(125, 200, 200, 1)


In [5]:
# Compute array of targets
nb_class = 5
Y_array = compute_targets_array(nb_class, X_train)
print(Y_array.shape)

# One-hot encode target column
Y_train = to_categorical(Y_array)
print(Y_train.shape)

(125,)
(125, 5)


### Building the model

In [6]:
# Create model
model = Sequential() # allows build a model layer by layer

# Add model layers

# Conv2D layer: 
# 64 nodes, 3x3 filter matrix, Rectified Linear Activation as activation function,
# shape of each input (200, 200, 1,) with 1 signifying images are greyscale
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(200,200,1))) 

# 32 nodes
model.add(Conv2D(32, kernel_size=3, activation='relu'))

# Flatten layer: connection between the convolution and dense layers
model.add(Flatten())

# Dense layer: used for the output layer
# 5 nodes for the output layer, one for each possible outcome (1-5)
# 'softmax' as activation function, it makes the output sump up to 1 so the output
# can be interpreted as probalities
model.add(Dense(5, activation='softmax'))

### Compiling the model

In [7]:
# Three parameters:
# optmizer: 'adam'
# loss function: 'categorical_crossentropy', the most common choice for classification
# metrics: 'accuracy', to see the accuracy score
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Training the model

In [8]:
%%time

# Number of epochs: number of tmes the model wil cycle trough the data
model.fit(X_train, Y_train, validation_data=(X_train, Y_train), epochs=30)

Train on 125 samples, validate on 125 samples
Epoch 1/30
125/125 [==============================] - 24s 189ms/step - loss: 1969604.1170 - accuracy: 0.2160 - val_loss: 1934728.2350 - val_accuracy: 0.2080
Epoch 2/30
125/125 [==============================] - 23s 183ms/step - loss: 1007733.9370 - accuracy: 0.1920 - val_loss: 162199.3170 - val_accuracy: 0.2000
Epoch 3/30
125/125 [==============================] - 24s 193ms/step - loss: 62347.1701 - accuracy: 0.2640 - val_loss: 1971.5147 - val_accuracy: 0.6880
Epoch 4/30
125/125 [==============================] - 23s 188ms/step - loss: 3407.0259 - accuracy: 0.6880 - val_loss: 299.7151 - val_accuracy: 0.8080
Epoch 5/30
125/125 [==============================] - 23s 188ms/step - loss: 153.1435 - accuracy: 0.8320 - val_loss: 1.8502 - val_accuracy: 0.9280
Epoch 6/30
125/125 [==============================] - 24s 191ms/step - loss: 0.8037 - accuracy: 0.9440 - val_loss: 0.4717 - val_accuracy: 0.9440
Epoch 7/30
125/125 [===========================

### Evalutation

In [9]:
# Evaluate the keras model
_, accuracy = model.evaluate(X_train, Y_train, verbose=0)
print('Accuracy: %.2f%%' % (accuracy*100))

Accuracy: 96.00%


### Make predictions

In [10]:
# Make probability predictions with the model
predictions = model.predict(X_train)

# Round predictions 
rounded = [round(x[0]) for x in predictions]

# Make class predictions with the model
predictions = model.predict_classes(X_train)

# Summarize the first 5 cases
for i in range(5):
    print('Predicted: %d (expected: %d)' % (predictions[i], Y_array[i]))

Predicted: 0 (expected: 0)
Predicted: 0 (expected: 0)
Predicted: 0 (expected: 0)
Predicted: 3 (expected: 0)
Predicted: 0 (expected: 0)


### Save weights and model

In [11]:
%%time

# Serialize model to JSON
model_json = model.to_json()
with open("output/neural_networks/model.json", "w") as json_file:
    json_file.write(model_json)
    
# Serialize weights to HDF5
model.save_weights("output/neural_networks/model.h5")
print("Saved model structure and weights")

Saved model structure and weights
Wall time: 241 ms


### Load model

In [12]:
# The model weights and architecture were saved separated, so it must re-compile

# Load json and create model
json_file = open('output/neural_networks/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# Load weights into new model
loaded_model.load_weights("output/neural_networks/model.h5")
print("Loaded model from disk")
 
# Evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
score = loaded_model.evaluate(X_train, Y_train, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
accuracy: 96.00%


### Summary

In [13]:
# Summarize model.
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 198, 198, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 196, 196, 32)      18464     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1229312)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 6146565   
Total params: 6,165,669
Trainable params: 6,165,669
Non-trainable params: 0
_________________________________________________________________


### Plot model

In [17]:
# Error, BUG, MUST FIX

# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)